In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import sklearn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/heart.csv')

In [5]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [6]:
X = df[['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']]
y = df[['target']]

In [7]:
X.shape

(1025, 13)

In [8]:
y.shape

(1025, 1)

In [ ]:
'''normalizing step'''
# X = (X - X.mean())/X.std()

'normalizing step'

In [9]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2


In [10]:
n_samples, n_feature = X.shape
n_s, n_labels = y.shape

print(n_samples, n_feature)
print(n_s, n_labels)

batch_size = 4

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state=51)

X_train = torch.tensor(X_train.astype(np.float32).values)
X_test = torch.tensor(X_test.astype(np.float32).values)
y_train = torch.tensor(y_train.astype(np.float32).values)
y_test = torch.tensor(y_test.astype(np.float32).values)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

1025 13
1025 1


In [11]:
y_train.shape

torch.Size([717, 1])

In [12]:
class Classification(nn.Module):
  def __init__(self,input_features,hidden_layer):
    super(Classification, self).__init__()
    self.l1 = nn.Linear(input_features, hidden_layer)
    self.l2 = nn.Linear(hidden_layer, 1)
    self.sigmoid = nn.Sigmoid()


  def forward(self, x):
    out = self.l1(x)
    out = F.relu(out)
    out = self.l2(out)
    out = self.sigmoid(out)

    return out

model = Classification(n_feature, n_feature)

In [13]:
def accuracy(loader, model):
  n_correct = 0
  n_sample = 0
  model.eval()

  with torch.no_grad():
    for input, output in loader:
      pred = model(input)
      pred = pred.round()
      n_correct = (pred == output).sum().item()
      n_sample = pred.shape[0]

    print(n_correct, n_sample)
    print(f'accuracy: {n_correct/n_sample}')

In [14]:
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
n_epochs = 100

for epoch in range(n_epochs):
  for i, data in enumerate(train_loader):
    input, output = data
    pred = model(input)
    loss = criterion(pred, output)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (i+1)%30 == 0:
      print(f'epoch: {epoch+1}/{n_epochs}, step: {i+1}/{len(train_loader)}, loss = {loss.item():.4f}')

epoch: 1/100, step: 30/180, loss = 0.8965
epoch: 1/100, step: 60/180, loss = 0.3915
epoch: 1/100, step: 90/180, loss = 0.8673
epoch: 1/100, step: 120/180, loss = 0.4214
epoch: 1/100, step: 150/180, loss = 1.4143
epoch: 1/100, step: 180/180, loss = 0.4092
epoch: 2/100, step: 30/180, loss = 1.3018
epoch: 2/100, step: 60/180, loss = 0.3762
epoch: 2/100, step: 90/180, loss = 0.1621
epoch: 2/100, step: 120/180, loss = 0.5613
epoch: 2/100, step: 150/180, loss = 0.3667
epoch: 2/100, step: 180/180, loss = 0.5908
epoch: 3/100, step: 30/180, loss = 0.8562
epoch: 3/100, step: 60/180, loss = 0.6910
epoch: 3/100, step: 90/180, loss = 0.4701
epoch: 3/100, step: 120/180, loss = 0.6215
epoch: 3/100, step: 150/180, loss = 1.0367
epoch: 3/100, step: 180/180, loss = 0.4863
epoch: 4/100, step: 30/180, loss = 1.1146
epoch: 4/100, step: 60/180, loss = 0.2223
epoch: 4/100, step: 90/180, loss = 0.3488
epoch: 4/100, step: 120/180, loss = 0.4934
epoch: 4/100, step: 150/180, loss = 0.2786
epoch: 4/100, step: 180

In [16]:
accuracy(test_loader, model)

4 4
accuracy: 1.0


In [17]:
pred = model(X_test)

In [18]:
n_correct = (pred.round() == y_test).sum().item()

In [21]:
print('Accuracy:', n_correct/len(X_test))

Accuracy: 0.8311688311688312
